In [30]:
# import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine 

In [31]:
# import list of recipes
csv_file = 'recipes.csv'
recipe_list = pd.read_csv(csv_file)
recipe_list.head()

,Name,#1,Material 1,#2,Material 2,#3,Material 3,#4,Material 4,#5,...,Sell,Miles Price,Source,Source Notes,Recipes to Unlock,Version,Category,Serial ID,Internal ID,Unique Entry ID
0,acorn pochette,6,acorn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,200,NaN,Balloons,Only available during Fall,0,1.0.0,Bags,2982,632,a9FFbP64fTcJ3Cn5E
1,acoustic guitar,8,softwood,3.0,iron nugget,NaN,NaN,NaN,NaN,NaN,...,200,NaN,Smug villagers,NaN,0,1.0.0,Housewares,406,44,Y62RymJYgbmZEyQon
2,angled signpost,2,hardwood,3.0,softwood,NaN,NaN,NaN,NaN,NaN,...,200,NaN,Lazy villagers,NaN,100,1.0.0,Housewares,440,510,BNiDaivRjGNzCckWw
3,apple chair,10,apple,4.0,wood,NaN,NaN,NaN,NaN,NaN,...,200,NaN,Big Sister villagers,NaN,100,1.0.0,Housewares,1700,189,HB55bMQf4D57ekNf4
4,apple dress,8,apple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,200,NaN,Big Sister villagers,NaN,100,1.0.0,Dresses,1782,621,KmfTvhh3qeyd6WcFJ


In [32]:
# clean dataframe
cleaned_recipe_list = recipe_list.iloc[:, 0:13].copy()
cleaned_recipe_list.head()

,Name,#1,Material 1,#2,Material 2,#3,Material 3,#4,Material 4,#5,Material 5,#6,Material 6
0,acorn pochette,6,acorn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,acoustic guitar,8,softwood,3.0,iron nugget,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,angled signpost,2,hardwood,3.0,softwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,apple chair,10,apple,4.0,wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,apple dress,8,apple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# import data for raw materials
materials_list = pd.read_csv('other.csv')
cleaned_materials_list = materials_list[['Name', 'Sell']]
cleaned_materials_list.head()

,Name,Sell
0,acorn,200.0
1,apple,500.0
2,apple tree,300.0
3,Aquarius fragment,500.0
4,arched tile path permit,NaN


In [34]:
# connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/acnh_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [35]:
engine.table_names()

/var/folders/6j/g7z_h15n2xb6lf5plnw73lrm0000gn/T/ipykernel_98705/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['recipe_list', 'materials_list']

In [36]:
# use pandas to load dataframe into database
cleaned_recipe_list.to_sql(name='recipe_list', con=engine, if_exists='append', index=False)
cleaned_materials_list.to_sql(name='materials_list', con=engine, if_exists='append', index=False)

In [37]:
# confirm data has been added
pd.read_sql_query('select * from recipe_list', con=engine).head()

,Name,#1,Material 1,#2,Material 2,#3,Material 3,#4,Material 4,#5,Material 5,#6,Material 6
0,acorn pochette,6,acorn,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
1,acoustic guitar,8,softwood,3.0,iron nugget,NaN,None,NaN,None,NaN,None,NaN,None
2,angled signpost,2,hardwood,3.0,softwood,NaN,None,NaN,None,NaN,None,NaN,None
3,apple chair,10,apple,4.0,wood,NaN,None,NaN,None,NaN,None,NaN,None
4,apple dress,8,apple,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None


In [38]:
pd.read_sql_query('select * from materials_list', con=engine).head()

,Name,Sell
0,acorn,200.0
1,apple,500.0
2,apple tree,300.0
3,Aquarius fragment,500.0
4,arched tile path permit,NaN
